# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops
from object_detection.builders import model_builder
from google.protobuf import text_format
import tensorflow as tf
import time

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')


/home/wc-gpu/MasterThesis/models/research/object_detection/utils/visualization_utils.py:27: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/wc-gpu/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/wc-gpu/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/wc-gpu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/wc-gpu/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/wc-gpu/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line

## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

/home/wc-gpu/MasterThesis/models/research/object_detection/utils/visualization_utils.py:27: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/wc-gpu/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/wc-gpu/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/wc-gpu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/wc-gpu/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/wc-gpu/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [4]:
# What model to download.
# MODEL_NAME = 'prod_faster_rcnn_resnet50_coco'
MODEL_NAME = 'faster_rcnn_resnet50_coco_2018_01_28'
# MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28'

MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')


# model_config="./prod_faster_rcnn_resnet50_coco/model.config"
model_config="./faster_rcnn_resnet50_coco_2018_01_28/model.config"
# model_config="./faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28/model.config"

## Download Model

In [5]:
# opener = urllib.request.URLopener()
# opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
# tar_file = tarfile.open(MODEL_FILE)
# for file in tar_file.getmembers():
#   file_name = os.path.basename(file.name)
#   if 'frozen_inference_graph.pb' in file_name:
#     tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [6]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [ ]:
from object_detection.protos import model_pb2
model_proto = model_pb2.DetectionModel()
with open(model_config, 'r') as myfile:
    config_string=myfile.read()
    text_format.Merge(config_string, model_proto)

In [12]:
detection_model = model_builder.build(model_config=model_proto,
                                      is_training=False)

NameError: name 'model_proto' is not defined

In [ ]:
tf_config = tf.ConfigProto()
tf_config.gpu_options.per_process_gpu_memory_fraction = 0.2

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## Helper code

In [7]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Extract proposal features

In [ ]:
def extract_region_proposal_features(image, model):
    tf.reset_default_graph()
   
    tf_image = tf.convert_to_tensor(np.expand_dims(image, 0), tf.float32)
    resized_image, true_image_shape = detection_model.preprocess(tf_image)
    rpn_features_to_crop, rett = model._feature_extractor.extract_proposal_features(
    resized_image, scope=detection_model.first_stage_feature_extractor_scope)
    init_op = tf.global_variables_initializer()

    with tf.Session(config=tf_config) as sess:
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
        sess.run(init_op)
        rpn_features_to_crop_out = sess.run(rpn_features_to_crop)
#         print (rpn_features_to_crop_out.shape, type(rpn_features_to_crop_out))
        coord.request_stop()
        coord.join(threads)
        return rpn_features_to_crop_out

In [ ]:
# SESSIONS_PATH = "/Users/mbaltac/Home/MasterThesis/MasterThesis/labeled_images_shopping_sessions/labeled"
SESSIONS_PATH = "/home/wc-gpu/storage4tb/session_data_thesis/sessions160000_165000"

In [ ]:
features = None
total_images = 0
sessions = os.listdir(SESSIONS_PATH)
actual_sessions = []

for session in sorted(sessions, reverse=True):
    try:
        a = int(session)
        session_path = os.path.join(SESSIONS_PATH, session)
        
        if os.path.exists(os.path.join(session_path, "annotations")):
            actual_sessions.append(session)
            images = [f for f in os.listdir(session_path) if f.endswith(".jpg")]
            total_images += len(images)
    except:
        continue

            
progress = 0

processed = 0
time_avg = 0
for session in sorted(actual_sessions, reverse=True):
    session_path = os.path.join(SESSIONS_PATH, session)
    
    images = [f for f in os.listdir(session_path) if f.endswith(".jpg")]
    for image_fname in sorted(images):
        features_path = os.path.join(session_path, "features")
        if not os.path.exists(os.path.join(features_path, image_fname+".feature.coco")):
            t0 = time.time()
            
            image_path = os.path.join(session_path, image_fname)
#             if not image_path.endswith(".jpg"):
#                 continue

            image = Image.open(image_path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.

            features = extract_region_proposal_features(image_np, detection_model)

            features_path = os.path.join(session_path, "features")
            if not os.path.exists(features_path):
                os.makedirs(features_path)

            with open(os.path.join(features_path, image_fname+".feature.coco"), "wb") as f:
                np.save(f, features)
            
            total_time = time_avg * processed
            processed += 1

            t1 = time.time()

            total = t1-t0
            print ("total ", total)
            
            time_avg = (total_time + total) / processed
            
#         else:
#             pass
#             print("skipped", os.path.join(features_path, image_fname+".feature.coco"))
        print ("progress", session, "  : ", progress / total_images, progress, total_images)
        print ("time", session, "  passed: ", time_avg*processed / 3600 ," h  expected left: ", time_avg* (total_images - progress) / 3600, " h")
        
        progress += 1
    

#         slc = features[0][:,:,500]
#         slc *= 255.0/slc.max()  

#         img = Image.fromarray(slc)
#         img = img.convert('RGB')

#         plt.figure(figsize=IMAGE_SIZE)
#         plt.imshow(image)

#         plt.figure(figsize=IMAGE_SIZE)
#         plt.imshow(img)

#         plt.figure(figsize=IMAGE_SIZE)
#         plt.imshow(image_np)
    

In [ ]:
def one_detection_features(features, xmin, ymin, xmax, ymax):
    final_feature = np.zeros(1024)
    xmin = xmin*features.shape[1]
    ymin = xmin*features.shape[0]
    xmax = xmin*features.shape[1]
    ymax = xmin*features.shape[0]
    for i in range(1024):
        final_feature[i] = np.max(features[0][ymin:ymax,xmin:xmax,i])
    
    return final_feature

In [ ]:
features[0].shape

In [ ]:
slc = features[0][:10,:,500]

In [ ]:
final_feature = np.zeros(1024)

In [ ]:
for i in range(1024):
    final_feature[i] = np.max(features[0][:,:,i])

In [ ]:
final_feature

In [ ]:
max(final_feature)

In [8]:
def run_inference_for_single_image(image, graph):
  with tf.Graph().as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
        
  return output_dict

In [9]:
TEST_IMAGE_PATHS = ["/home/wc-gpu/MasterThesis/session_data/122923_testing/"+f for f in os.listdir("/home/wc-gpu/MasterThesis/session_data/122923_testing") if f.endswith("jpg")]

In [10]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [11]:
features = None
for image_path in sorted(TEST_IMAGE_PATHS):
    print (image_path)
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
   
    output_dict, features = run_inference_for_single_image(image_np, detection_model)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=2)
    slc = features[0][:,:,500]
    slc *= 255.0/slc.max()  

    img = Image.fromarray(slc)
    img = img.convert('RGB')

    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image)

    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(img)

    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)
    break

/home/wc-gpu/MasterThesis/session_data/122923_testing/ecenter_porta_122923_1530531480.913474_TOP_FRONT_EventResolved.jpg


NameError: name 'detection_model' is not defined